In [2]:
siteUrl = "https://www.rvtrader.com/search-results"
siteUrl = "?".join([siteUrl, "sort=featured%3Aasc"])
siteUrl = "&".join([siteUrl, "radius=150"])
siteUrl = "&".join([siteUrl, "seller_type=NT%2CCSNT%2CPM"])
siteUrl = "&".join([siteUrl, "layoutView=listView"])
print(siteUrl)

proxies = {'http': 'http://<<USERID>>:<<PASSWORD>>@23.19.51.245:40791/'}

https://www.rvtrader.com/search-results?sort=featured%3Aasc&radius=150&seller_type=NT%2CCSNT%2CPM&layoutView=listView


In [4]:
import requests
from bs4 import BeautifulSoup

# Retrieve the page with tag results and set it up to be scraped
sitePage = requests.get(url=siteUrl, proxies=proxies)
siteSoup = BeautifulSoup(sitePage.content, 'lxml')

#<a class="btn pageJump btn-inverse" data-track="Search - Pagination - Bottom - Last Link" data-page="218"
#>Last<span class="hidden-xs hidden-md"> Page</span></a>
siteLinks = siteSoup.find_all('a', class_='btn pageJump btn-inverse')

# Get the max page link
max_page = 0
if len(siteLinks):
    max_page = int(siteLinks[0]['data-page'])
print(max_page)

218


In [5]:
from random import randint

i = randint(1, max_page)
tagUrl = "&".join([siteUrl, "page="+str(i)])
print(tagUrl)

tagPage = requests.get(url=tagUrl, proxies=proxies)
tagSoup = BeautifulSoup(tagPage.content, 'lxml')

# Get the title of the page to prove we are progressing
tagTitle = tagSoup.select("head > title")
titleString = tagTitle[0].get_text().strip()
print(titleString)

https://www.rvtrader.com/search-results?sort=featured%3Aasc&radius=150&seller_type=NT%2CCSNT%2CPM&layoutView=listView&page=56
Page 56 of 218 - New or Used RVs For Sale - Fleetwood, Airstream, Winnebago, Keystone, Forest River, Newmar - RVTrader.com


In [7]:
import urllib.parse
import re

seen_urls_array = []

#<a data-dlrid="" data-custid="10372090" data-adid="120364049" data-track="Show Phone Number Click" 
#class="callButton"><i class="fa fa-phone"></i> Show Phone Number</a>
phoneLinks = tagSoup.select('a[data-track="Show Phone Number Click"]')

for phoneLink in phoneLinks:
    
    phoneDiv = phoneLink.parent.parent.parent.parent.parent
    #print(phoneDiv)

    # Get the links to the individual pages
    pageLinks = phoneDiv.select('a[data-dlrid="private seller"]')
    for pageLink in pageLinks:
        pageUrl = urllib.parse.urljoin(tagUrl, re.sub(' ', '%20', pageLink['href']))
        if pageUrl not in seen_urls_array:
            seen_urls_array.append(pageUrl)

for pageUrl in seen_urls_array:
    print(pageUrl)

https://www.rvtrader.com/listing/2011-Tiffin-Motorhomes-ALLEGRO-RED-38QBA-120819449
https://www.rvtrader.com/listing/2011-Gulf-Stream-Endura-Max-MAX-3912-118814062
https://www.rvtrader.com/listing/2011-Jayco-GREYHAWK-31FS-120750297
https://www.rvtrader.com/listing/2011-American-Coach-AMERICAN-EAGLE-117686630
https://www.rvtrader.com/listing/2011-Tiffin-Motorhomes-Allegro-Bus-43QBP-117361402


In [8]:
seen_urls_array = []
for i in range(1, max_page+1):
    tagUrl = "&".join([siteUrl, "page="+str(i)])
    #print(tagUrl)

    tagPage = requests.get(url=tagUrl, proxies=proxies)
    tagSoup = BeautifulSoup(tagPage.content, 'lxml')

    # Get the title of the page to prove we are progressing
    tagTitle = tagSoup.select("head > title")
    titleString = tagTitle[0].get_text().strip()
    #print(titleString)

    phoneLinks = tagSoup.select('a[data-track="Show Phone Number Click"]')
    for phoneLink in phoneLinks:
        phoneDiv = phoneLink.parent.parent.parent.parent.parent
        #print(phoneDiv)

        # Get the links to the individual pages
        pageLinks = phoneDiv.select('a[data-dlrid="private seller"]')
        for pageLink in pageLinks:
            pageUrl = urllib.parse.urljoin(tagUrl, re.sub(' ', '%20', pageLink['href']))
            #print(pageUrl)
            if pageUrl not in seen_urls_array:
                seen_urls_array.append(pageUrl)
    
    # Destroy the tree when you're done working with it
    tagSoup.decompose()

# Show the number of individual page links you have
print(len(seen_urls_array))

1236


In [9]:
import pickle

with open('rvtrader.pickle', 'wb') as handle:
    pickle.dump(seen_urls_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle

with open('rvtrader.pickle', 'rb') as handle:
    seen_urls_array = pickle.load(handle)

In [10]:
from selenium import webdriver
from base64 import b64encode

proxy = {'host': '23.19.51.245', 'port': '40791', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>'}
# Retrieve the page with tag results and set it up to be scraped
fp = webdriver.FirefoxProfile()
fp.set_preference('network.proxy.type', 1)
fp.set_preference('network.proxy.http', proxy['host'])
fp.set_preference('network.proxy.http_port', int(proxy['port']))
fp.set_preference('network.proxy.no_proxies_on', 'localhost, 127.0.0.1')
fp.add_extension('closeproxy.xpi')
credentials = '{usr}:{pwd}'.format(**proxy)
credentials = b64encode(credentials.encode('ascii')).decode('utf-8')
fp.set_preference('extensions.closeproxyauth.authtoken', credentials)

#set some privacy settings
fp.set_preference( "places.history.enabled", False )
fp.set_preference( "privacy.clearOnShutdown.offlineApps", True )
fp.set_preference( "privacy.clearOnShutdown.passwords", True )
fp.set_preference( "privacy.clearOnShutdown.siteSettings", True )
fp.set_preference( "privacy.sanitize.sanitizeOnShutdown", True )
fp.set_preference( "signon.rememberSignons", False )
fp.set_preference( "network.cookie.lifetimePolicy", 2 )
fp.set_preference( "network.dns.disablePrefetch", True )
fp.set_preference( "network.http.sendRefererHeader", 0 )

#if you're really hardcore about your security
#js can be used to reveal your true i.p.
fp.set_preference( "javascript.enabled", False )

#get a huge speed increase by not downloading images
fp.set_preference( "permissions.default.image", 2 )

In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import zipfile

manifest_json = """
{
    "version": "1.0.0",
    "manifest_version": 2,
    "name": "Chrome Proxy",
    "permissions": [
        "proxy",
        "tabs",
        "unlimitedStorage",
        "storage",
        "<all_urls>",
        "webRequest",
        "webRequestBlocking"
    ],
    "background": {
        "scripts": ["background.js"]
    },
    "minimum_chrome_version":"22.0.0"
}
"""

background_js = """
var config = {
        mode: "fixed_servers",
        rules: {
          singleProxy: {
            scheme: "http",
            host: "23.19.51.245",
            port: parseInt(40791)
          },
          bypassList: ["foobar.com"]
        }
      };

chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

function callbackFn(details) {
    return {
        authCredentials: {
            username: "<<USERID>>",
            password: "<<PASSWORD>>"
        }
    };
}

chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {urls: ["<all_urls>"]},
            ['blocking']
);
"""


pluginfile = 'proxy_auth_plugin.zip'

with zipfile.ZipFile(pluginfile, 'w') as zp:
    zp.writestr("manifest.json", manifest_json)
    zp.writestr("background.js", background_js)

co = Options()
co.add_argument("--start-maximized")
co.add_argument('--dns-prefetch-disable')
#co.add_argument('--disable-extensions')
co.add_extension(pluginfile)

In [33]:
driver = webdriver.Chrome(chrome_options=co)

#driver = webdriver.Firefox(fp)

In [34]:
from random import randint

pageUrl = seen_urls_array[randint(0,len(seen_urls_array)-1)]
print(pageUrl)

https://www.rvtrader.com/listing/2014-Grand-Design-MOMENTUM-385TH-120038994


In [35]:
import time

# set timeout information
driver.set_page_load_timeout(50)

finished = 0
fails = 0
while finished == 0 and fails < 8:
    try:
        driver.get(pageUrl)
        finished = 1
    except Exception as e:
        print(str(e))
        fails = fails + 1
        time.sleep(5)

In [31]:
driver.quit()

In [36]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    buttonCss = "div.seller-info-telephone > button"
    
    # Wait for button to show up
    buttonTag = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, buttonCss))
    )
    
    buttonTag.click()
except Exception as e:
    print(str(e))
#driver.save_screenshot('cycletrader.png')

In [37]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

phoneRe = re.compile(r"\(?[0-9]{3}\)?[-. ]?[0-9]{3}[-. ]?[0-9]{4}")

#<div class="seller-info-telephone">&nbsp;-&nbsp;<i class="fa fa-phone"></i> 1-574-971-6834</div>
phoneCss = 'div.sellerOverview > div.seller-info-telephone'

# Wait for button to disappear
#print(driver.find_element_by_css_selector(buttonCss).get_attribute('outerHTML'))
WebDriverWait(driver, 30).until_not(
    EC.visibility_of_element_located((By.CSS_SELECTOR, buttonCss))
)

phoneHTML = driver.find_element_by_css_selector(phoneCss).get_attribute('innerHTML')
#print(phoneHTML)
match = phoneRe.search(phoneHTML)
if match:
    phone = match.group()
else:
    phone = ""
print(phone)

<button data-dlrid="" data-custid="10273732" data-adid="120038994" data-track="Show Phone Number Click" class="btn btn-primary callButton hidden-xs hidden-md hidden-sm"><i class="fa fa-phone"></i> <span>Show Phone Number</span></button>
&nbsp;-&nbsp;<i class="fa fa-phone"></i> 1-574-971-6834
574-971-6834


In [38]:
try:

    model = driver.find_element_by_css_selector("div.detail-title.lfloat > h1").text.strip()
    if len(model):
        model = re.sub(r'\s+', ' ', model)
    else:
        model = "Unknown"
    print(model)

    price = driver.find_element_by_css_selector("div.detail-price.rfloat > h2 > span").text.strip()
    if not len(price):
        price = "Unknown"
    print(price)

except Exception as e:
    print(str(e))
finally:
    driver.quit()

2014 GRAND DESIGN MOMENTUM 385TH
$55,900


In [39]:
import time
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

phoneRe = re.compile(r"\(?[0-9]{3}\)?[-. ]?[0-9]{3}[-. ]?[0-9]{4}")
for pageUrl in seen_urls_array:
    try:
        driver = webdriver.Chrome(chrome_options=co)

        # Set timeout information
        driver.set_page_load_timeout(50)

        # Bring up the page
        finished = 0
        fails = 0
        while finished == 0 and fails < 8:
            try:
                driver.get(pageUrl)
                finished = 1
            except Exception as e:
                print(str(e))
                fails = fails + 1
                time.sleep(5)

        # Click the phone button
        try:
            buttonCss = "div.seller-info-telephone > button"

            # Wait for button to show up
            buttonTag = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, buttonCss))
            )

            buttonTag.click()
        except Exception as e:
            print(str(e))

        # Get the phone number
        phoneCss = 'div.sellerOverview > div.seller-info-telephone'

        # Wait for button to disappear
        WebDriverWait(driver, 30).until_not(
            EC.visibility_of_element_located((By.CSS_SELECTOR, buttonCss))
        )

        phoneHTML = driver.find_element_by_css_selector(phoneCss).get_attribute('innerHTML')
        match = phoneRe.search(phoneHTML)
        if match:
            phone = match.group()
        else:
            phone = ""
        #print(phone)

        if len(phone):

            model = driver.find_element_by_css_selector("div.detail-title.lfloat > h1").text.strip()
            if len(model):
                model = re.sub(r'\s+', ' ', model)
            else:
                model = "Unknown"
            #print(model)

            price = driver.find_element_by_css_selector("div.detail-price.rfloat > h2 > span").text.strip()
            if not len(price):
                price = "Unknown"
            #print(price)

            bigFile = open('rvtrader.txt', 'a', encoding='utf-8')
            bigFile.write(model + '\t' + price + '\t' + phone + '\n')
            bigFile.close()

        else:
            print(phoneHTML)
            driver.save_screenshot('rvtrader.png')

    except Exception as e:
        print(str(e))
    finally:
        driver.quit()

&nbsp;-&nbsp;<i class="fa fa-phone"></i> null
&nbsp;-&nbsp;<i class="fa fa-phone"></i> null
&nbsp;-&nbsp;<i class="fa fa-phone"></i> null
&nbsp;-&nbsp;<i class="fa fa-phone"></i> null
&nbsp;-&nbsp;<i class="fa fa-phone"></i> null
&nbsp;-&nbsp;<i class="fa fa-phone"></i> null
&nbsp;-&nbsp;<i class="fa fa-phone"></i> null
&nbsp;-&nbsp;<i class="fa fa-phone"></i> null
&nbsp;-&nbsp;<i class="fa fa-phone"></i> null
&nbsp;-&nbsp;<i class="fa fa-phone"></i> null
&nbsp;-&nbsp;<i class="fa fa-phone"></i> null
Message: 

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.sellerOverview > div.seller-info-telephone"}
  (Session info: chrome=56.0.2924.87)
  (Driver info: chromedriver=2.27.440174 (e97a722caafc2d3a8b807ee115bfb307f7d2cfd9),platform=Windows NT 10.0.14393 x86_64)

&nbsp;-&nbsp;<i class="fa fa-phone"></i> null
&nbsp;-&nbsp;<i class="fa fa-phone"></i> null
&nbsp;-&nbsp;<i class="fa fa-phone"></i> null
Message: 

&nbsp;-&nbsp;<i class="fa fa-phone